In [1]:
import string 
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import matplotlib.pyplot as plt

from keras.applications.inception_v3 import InceptionV3, preprocess_input 
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model

# small library for seeing the progress of loops.
#from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
tqdm().pandas()

0it [00:00, ?it/s]/usr/local/lib/python3.7/site-packages/tqdm/std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
PathText = '/Users/anjalisingh/Documents/ImageCaptionGenerator/Description_of_images/Flickr8k.token.txt'

In [3]:
imagePath = '/Users/anjalisingh/Documents/ImageCaptionGenerator/Flicker8k_Dataset'

In [4]:
# Loading a text file into memory Opening the file as read only
def load_doc(PathText):
    file = open(PathText, 'r')
    text = file.read()
    file.close()
    return text

TokenText = load_doc(PathText)

In [5]:
# get all imgs with their captions 
def all_img_captions(PathText):
    captions = TokenText.split('\n')
    descriptions = {}
    for caption in captions[:-1]:          # all the captions in the list captions so we triversing through it
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:    
            # img contains name of the img#captionNumber caption number
            # represents the numberth of the caption of a particular image 
            # as an image contains 5 captions each. 
            # img[from starting: every character except for last 2 (#captionNumber)]
            descriptions[img[:-2]] = [caption]
        else:
            # arranging caption and img name accordingly 
            descriptions[img[:-2]].append(caption)
    return descriptions 

descriptions = all_img_captions(TokenText)

0it [00:00, ?it/s]


In [6]:
print("Length of descriptions =", len(descriptions))

Length of descriptions = 8092


In [7]:
def cleaning_text(captions):
    table = str.maketrans('','', string.punctuation)
    for img, caps in captions.items():
        for i, img_caption in enumerate(caps):   
            # enumerate takes all the 5 captions of a particular 
            # image and gives then index from 0 to 4
            img_caption.replace("-"," ")
            desc = img_caption.split()
            # converts to lowercase
            desc = [word.lower() for word in desc]
            # remove punctuation from each token 
            desc = [word.translate(table) for word in desc]
            # remove hanging 's and a
            desc = [word for word in desc if(len(word)>1)]
            # remove tokens with numbers in them
            desc = [word for word in desc if(word.isalpha())]
            # convert back to string           
            img_caption = ' '.join(desc)

            captions[img][i] = img_caption
    return captions

clean_description = cleaning_text(descriptions)

In [8]:
print("Length of descriptions =", len(clean_description))

Length of descriptions = 8092


In [9]:
# build vocabulary of all the unique words
def text_vocabulary(descriptions):
    vocab = set()    # using set because it won't allow duplicate values
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab

vocabulary = text_vocabulary(clean_description)

In [10]:
def save_descriptions(descriptions, PathText):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc)
        data = "\n".join(lines)
        file = open(PathText, "w")
        file.write(data)
        file.close()


save_descriptions(clean_description, "descriptions.txt")

In [11]:
def extract_features_In(directory):
    model_In = InceptionV3(include_top=False, weights='imagenet')
    features = {}
    for img in tqdm(os.listdir(directory)):
        filename = directory + "/"+img
        image = Image.open(filename)
        image = image.resize((299, 299))
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        
        feature = model_In.predict(image)
        features[img] = feature
    return features 

# features_In = extract_features_In(imagePath)
# dump(features_In, open("features_In.p", "wb"))
features_In = load(open("features_In.p", "rb"))

In [12]:
# os.listdir(imagePath)

In [13]:
for img in os.listdir(imagePath):
    filename = imagePath + "/" + img
#     print(filename)

In [14]:
trainText = '/Users/anjalisingh/Documents/ImageCaptionGenerator/Description_of_images/Flickr_8k.trainImages.txt'

In [15]:
# load the data
def load_photos(filename):
    file = load_doc(filename)
    photos = file.split("\n")[:-1]
    return photos
train_imgs = load_photos(trainText)

In [16]:
# description.txt contains all the image name and description
# Now, by using this function we will take out the ones on which we need to train our model
def load_clean_descriptions(filename, photos):
    # loading clean_descriptions
    file = load_doc(filename)
    descriptions = {}
    for line in file.split("\n"):
        words = line.split()
        if len(words)<1:
            continue
        
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            
            desc = '<start>' + " ".join(image_caption) + '<end>'
            descriptions[image].append(desc)
            
    return descriptions

train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)

In [17]:
def load_features_In(photos):
    all_features = load(open("features_In.p", "rb"))
    features_In = {k:all_features[k] for k in photos}
    return features_In

train_features_In = load_features_In(train_imgs)

In [18]:
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

desc_list = dict_to_list(descriptions)
# desc_list

In [19]:
def create_tokenizer_In(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

tokenizer = create_tokenizer_In(train_descriptions)
dump(tokenizer, open('tokenizer_In.p', 'wb'))
vocab_size = len(tokenizer.word_index) + 1
vocab_size

7577

In [20]:
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)

max_length = max_length(descriptions)
max_length

32

In [ ]:
X1, X2, y = list(), list(), list()
# walk through each description for the image
for key, desc_list in train_descriptions.items():
    feature = features_In[key][0]
        # encode the sequence
    for cap in desc_list:
        seq = tokenizer.texts_to_sequences([cap])[0]
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)

            
# kernel died
X2 = np.array(X2)
X1 = np.array(X1)
y = np.array(y)


In [ ]:
# features from the CNN model squeezed from 2048 to 256 nodes
inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
    
# LSTM sequence Model
inputs2 = Input(shape=(4096,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
    
# Merging both models
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
# tie it together [image, seq][word]
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit([X1, X2], y, epochs = 20, batch_size = 256) 
    
# summarize model
print(model.summary())